In [ ]:
#import deps

import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from pybaseball import playerid_reverse_lookup
from pybaseball import statcast
from pybaseball import playerid_lookup
import openpyxl
import pickle
from pathlib import Path
import os
import re
import time
from datetime import datetime, timedelta
from pybaseball import statcast_single_game, schedule_and_record, pitching_stats_range, batting_stats_range, statcast_pitcher

In [ ]:
datesDict = {
    "2010": {'404'},
    "2011": {'331'},
    "2012": {'328'},
    "2013": {'331'},
    "2014": {'322'},
    "2015": {'405'},
    "2016": {'403'},
    "2017": {'402'},
    "2018": {'329'},
    "2019": {'320'},
    "2020": {'723'},
    "2021": {'401'}
}

def fetchGameDates(sh, key):
    for row in sh.iter_rows(min_row=2, min_col=1, max_row=None, max_col=1):
        datesDict[str(key)].add(row[0].value)


for index in range(12):
    # load excel with its path
    path = "mlb-odds-" + str(2010 + index) + ".xlsx"
    wrkbk = openpyxl.load_workbook(path)

    sh = wrkbk.active
    fetchGameDates(sh, 2010 + index)

datesDict["2010"] = sorted([int(x) for x in datesDict["2010"]])
datesDict["2011"] = sorted([int(x) for x in datesDict["2011"]])
datesDict["2012"] = sorted([int(x) for x in datesDict["2012"]])
datesDict["2013"] = sorted([int(x) for x in datesDict["2013"]])
datesDict["2014"] = sorted([int(x) for x in datesDict["2014"]])
datesDict["2015"] = sorted([int(x) for x in datesDict["2015"]])
datesDict["2016"] = sorted([int(x) for x in datesDict["2016"]])
datesDict["2017"] = sorted([int(x) for x in datesDict["2017"]])
datesDict["2018"] = sorted([int(x) for x in datesDict["2018"]])
datesDict["2019"] = sorted([int(x) for x in datesDict["2019"]])
datesDict["2021"] = sorted([int(x) for x in datesDict["2021"]])

datesDict["2010"].pop(0)
datesDict["2011"].pop(0)
datesDict["2012"].pop(0)
datesDict["2013"].pop(0)
datesDict["2014"].pop(0)
datesDict["2015"].pop(0)
datesDict["2016"].pop(0)
datesDict["2017"].pop(0)
datesDict["2018"].pop(0)
datesDict["2019"].pop(0)
datesDict["2021"].pop(0)

print(datesDict["2010"])
print(datesDict["2011"])
print(datesDict["2012"])
print(datesDict["2013"])
print(datesDict["2014"])
print(datesDict["2015"])
print(datesDict["2016"])
print(datesDict["2017"])
print(datesDict["2018"])
print(datesDict["2019"])
print(datesDict["2021"])

In [ ]:
for key in datesDict:
    print(key)
    for date in datesDict[key]:
        date = str(date)
        dayDf=pd.DataFrame([])
        modMonth = date[:-2]
        if len(modMonth) == 1:
            modMonth = "0" + modMonth
        day=key + '-' + modMonth + '-' + date[-2:]
        temp=pd.DataFrame([])
        try:
            temp=statcast(day,day)
        except:
            print(day)
        if len(temp)>0:
            dayDf=pd.DataFrame(temp)
            fileName = key + '_every_pitch_' + modMonth + '_' + date[-2:] + '.pkl'
            with open(fileName, 'wb') as f:  # open a text file
                pickle.dump(dayDf, f) # serialize the data frame

    #every_pitch=combined pickle deserialized files
    #pickle_in=open("2010_every_pitch_04_04.pkl","rb")
    #every_pitch=pickle.load(pickle_in)
    #print(every_pitch)


In [ ]:
listOfEveryPitchFilenames = []
path = "./"
pattern = r"\d{4}_every_pitch_\d{2}_\d{2}\.pkl$"  # Match all every_pitch file

for filename in os.listdir(path):
    if re.search(pattern, filename):
        listOfEveryPitchFilenames.append(filename)
print(len(listOfEveryPitchFilenames))

game_stats_fails = []
for fileName in listOfEveryPitchFilenames:
    today_games_pickle_in=open(fileName,"rb")
    today_games=pickle.load(today_games_pickle_in)
    game_ids=today_games.game_pk.unique()
    try:
        game_ids=game_ids.astype(int)
    except:
        print("invalid game ids for day: " + fileName)
        game_stats_fails.append(pd.DataFrame({'fileName':fileName,'Home':0,'Away':0,
                 'Reason':'invalid game ids for day: ' + fileName},index=[0]))
        continue
    date = fileName[:4] + "-" + fileName[-9:-7] + "-" + fileName[-6:-4]
    for game in game_ids:
        print(game)
        print(date)
        temp_game_stats=pd.DataFrame([])
        temp_game_stats_fn = fileName[:4] + "_game_stats_" + fileName[-9:-7] + "_" + fileName[-6:-4] + "_" + str(game) + ".pkl"
        temp_game_stats_pickle_in = pd.DataFrame([])
        try:
            temp_game_stats_pickle_in = open(temp_game_stats_fn,"rb")
            temp_game_stats_pickle_in=pickle.load(temp_game_stats_pickle_in)
        except:
            print(temp_game_stats_fn + " not found, creating it")
            temp_game_stats_pickle_in = pd.DataFrame([])

        if len(temp_game_stats_pickle_in) == 0:
            try:
                temp_game_stats=statcast_single_game(game)
            except:
                print('game data pull failed')
                game_stats_fails.append(pd.DataFrame({'game#':game,'Date':date,'Home':0,'Away':0,
                                                 'Reason':'game data pull failed'},index=[0]))
                continue
            if len(temp_game_stats)>0:
                temp_game_stats_df=pd.DataFrame(temp_game_stats)
                with open(temp_game_stats_fn, 'wb') as f:  # open a text file
                    pickle.dump(temp_game_stats_df, f) # serialize the data frame
            elif len(temp_game_stats)==0:
                print('No game stats')
                game_stats_fails.append(pd.DataFrame({'game#':game,'Date':date,'Home':0,'Away':0,
                                                 'Reason':'no game data'},index=[0]))
                continue
            else:
                game_stats_fails.append(pd.DataFrame({'game#':game,'Date':date,'Home':0,'Away':0,
                                 'Reason':'unknown'},index=[0]))
                continue
        else:
            print(temp_game_stats_fn + " already exists")
            continue

In [ ]:
listOfGameStatsFilenames = []
path = "./"
gameStatsPattern = r"\d{4}_game_stats_\d{2}_\d{2}_\d+\.pkl$"  # Match all game_stats file

for filename in os.listdir(path):
    if re.search(gameStatsPattern, filename):
        listOfGameStatsFilenames.append(filename)
print("listOfGameStatsFilenames length")
print(len(listOfGameStatsFilenames))

schedule_record_fails = []

for fileName in listOfGameStatsFilenames:
    # In different databases, there are a couple teams with different abbreviations: this handles that.
    game_stats = open(fileName,"rb")
    game_stats=pickle.load(game_stats)
    home,away=game_stats.home_team[0],game_stats.away_team[0]
    date=fileName[:4] + "-" + fileName[16:18] + "-" + fileName[19:21]
    if home=='CWS':
        home='CHW'
    else:
        None
    if away=='CWS':
        away='CHW'
    else:
        None
    if home=='WSH':
        home='WSN'
    else:
        None
    if away=='WSH':
        away='WSN'
    else:
        None
    print(home,away)

    # Harrison check
    # Regular expression pattern to match consecutive digits before ".pkl"
    patternForGameNumber = r'(\d+)\.pkl$'

    # Search for the pattern in the filename
    gameMatch = re.search(patternForGameNumber, fileName)

    game = ""

    # If a match is found, extract the consecutive digits
    if gameMatch:
        game = gameMatch.group(1)
    else:
        print("No match found for game number in file name ", fileName)
        continue

    temp_home_schedule_record_fn = fileName[:4] + "_game_stats_" + fileName[16:18] + "_" + fileName[19:21] + "_" + game + "_home_schedule_record.pkl"
    temp_away_schedule_record_fn = fileName[:4] + "_game_stats_" + fileName[16:18] + "_" + fileName[19:21] + "_" + game + "_away_schedule_record.pkl"

    temp_home_schedule_record_pickle_in = pd.DataFrame([])
    temp_away_schedule_record_pickle_in = pd.DataFrame([])
    temp_home_schedule_record = pd.DataFrame([])
    temp_away_schedule_record = pd.DataFrame([])
    try:
        temp_home_schedule_record_pickle_in = open(temp_home_schedule_record_fn,"rb")
        temp_home_schedule_record_pickle_in=pickle.load(temp_home_schedule_record_pickle_in)
        temp_away_schedule_record_pickle_in = open(temp_away_schedule_record_fn,"rb")
        temp_away_schedule_record_pickle_in=pickle.load(temp_away_schedule_record_pickle_in)
    except:
        print(temp_home_schedule_record_fn + "_" + temp_away_schedule_record_fn + " not found, creating it")
        temp_home_schedule_record_pickle_in = pd.DataFrame([])
        temp_away_schedule_record_pickle_in = pd.DataFrame([])

    if len(temp_home_schedule_record_pickle_in) == 0 or len(temp_away_schedule_record_pickle_in) == 0:
        try:
            temp_home_schedule_record=schedule_and_record(pd.to_datetime(date).year,home)
            temp_away_schedule_record=schedule_and_record(pd.to_datetime(date).year,away)
        except:
            print('schedule_and_record failed ',date, home, away)
            schedule_record_fails.append(pd.DataFrame({'game#':game,'Date':date,'Home':home,'Away':away,
                                             'Reason':'schedule_and_record failed'},index=[0]))
            continue
        if len(temp_home_schedule_record)>0 and len(temp_away_schedule_record)>0:
            with open(temp_home_schedule_record_fn, 'wb') as f:  # open a text file
                pickle.dump(temp_home_schedule_record, f) # serialize the data frame
            with open(temp_away_schedule_record_fn, 'wb') as f:  # open a text file
                pickle.dump(temp_away_schedule_record, f) # serialize the data frame
        elif len(temp_home_schedule_record)==0 or len(temp_away_schedule_record)==0:
            print('No home_schedule_record or away_schedule_record')
            schedule_record_fails.append(pd.DataFrame({'game#':game,'Date':date,'Home':home,'Away':away,
                                             'Reason':'schedule_and_record failed'},index=[0]))
            continue
        else:
            schedule_record_fails.append(pd.DataFrame({'game#':game,'Date':date,'Home':home,'Away':away,
                                             'Reason':'schedule_and_record failed'},index=[0]))
            continue

    else:
        print(temp_home_schedule_record_fn + " and " + temp_away_schedule_record_fn + " already exists")
        continue


In [ ]:
listOfYears = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
listOfTeams = ['CHW', 'WSN', 'NYY', 'LAA', 'LAD', 'ARI', 'SD', 'ATL', 'FLA', 'CIN', 'DET', 'KC', 'CHC', 'MIL', 'COL', 'NYM', 'SEA', 'OAK', 'BAL', 'TB', 'CLE', 'TEX', 'PHI', 'MIN', 'BOS', 'SF', 'PIT', 'TOR', 'STL', 'HOU']

for year in listOfYears:
    for team in listOfTeams:
        if year >= 2012 and team == 'FLA':
            team = 'MIA'
        temp_home_schedule_record=schedule_and_record(year,team)
        temp_schedule_record_fn = str(year) + "_schedule_record_" + team + ".pkl"
        with open(temp_schedule_record_fn, 'wb') as f:  # open a text file
            pickle.dump(temp_home_schedule_record, f) # serialize the data frame


print("completed fetching schedule_and_record")

In [ ]:
path = "./"
scheduleRecordPattern = r"\d{4}_schedule_record_[a-zA-Z]+\.pkl$"  # Match all every_pitch file
scheduleRecordFileNames = []

for filename in os.listdir(path):
    if re.search(scheduleRecordPattern, filename):
        scheduleRecordFileNames.append(filename)
print("scheduleRecordFileNames length")
print(len(scheduleRecordFileNames))


In [ ]:
print("run all_pitching_stats")
everyPitchFilenames = []
path = "./"
everyPitchPattern = r"\d{4}_every_pitch_\d{2}_\d{2}\.pkl$"  # Match all game_stats file

currentYear = "0"
for filename in os.listdir(path):
    if re.search(everyPitchPattern, filename):
        if currentYear == filename[:4]:
            everyPitchFilenames.append(filename)
        else:
            currentYear = filename[:4]

print("everyPitchFilenames")
print(everyPitchFilenames)
print(len(everyPitchFilenames))
filenameFormatNoExists = []
pitching_stats_range_fails = []

for filename in everyPitchFilenames:
    # Define the date in YYYY-MM-DD format

    # Extract year, month, and day from the filename using regular expressions
    match = re.search(r'(\d{4})_every_pitch_(\d{2})_(\d{2})\.pkl', filename)
    if match:
        year = match.group(1)
        month = match.group(2)
        day = match.group(3)

        # Create a datetime object
        date_obj = datetime(int(year), int(month), int(day))

        # Format the date as "YYYY-MM-DD"
        date_string = date_obj.strftime("%Y-%m-%d")

        # Convert the date string to a datetime object
        date = datetime.strptime(date_string, "%Y-%m-%d")
        # Calculate the date 30 days prior
        prior_date = date - timedelta(days=29)
        # Format the prior_date as YYYY-MM-DD
        prior_date_formatted = prior_date.strftime("%Y-%m-%d")
        print(date_string)
        print(prior_date_formatted)

        temp_all_pitching_stats=pd.DataFrame([])
        temp_all_pitching_stats_fn = filename.replace("_every_pitch_", "_all_pitching_stats_")
        temp_all_pitching_stats_pickle_in = pd.DataFrame([])
        try:
            temp_all_pitching_stats_pickle_in = open(temp_all_pitching_stats_fn,"rb")
            temp_all_pitching_stats_pickle_in=pickle.load(temp_all_pitching_stats_pickle_in)
        except:
            print(temp_all_pitching_stats_fn + " not found, creating it")
            temp_all_pitching_stats_pickle_in = pd.DataFrame([])
        if len(temp_all_pitching_stats_pickle_in) == 0:
            try:
                time.sleep(5)
                temp_all_pitching_stats = pitching_stats_range(prior_date_formatted, date.strftime("%Y-%m-%d"))
            except Exception as e:
                print('pitching_stats_range failed ',date.strftime("%Y-%m-%d"), e, prior_date_formatted)
                pitching_stats_range_fails.append(pd.DataFrame({'Date':date,'prior_date_formatted':prior_date_formatted,'exception':e,'Reason':'pitching_stats_range failed'},index=[0]))
                continue
            if len(temp_all_pitching_stats)>0:
                with open(temp_all_pitching_stats_fn, 'wb') as f:  # open a text file
                    pickle.dump(temp_all_pitching_stats, f) # serialize the data frame
            elif len(temp_all_pitching_stats)==0:
                print('No temp_all_pitching_stats')
                pitching_stats_range_fails.append(pd.DataFrame({'Date':date,'Reason':'pitching_stats_range failed'},index=[0]))
                continue
            else:
                pitching_stats_range_fails.append(pd.DataFrame({'Date':date,'Reason':'pitching_stats_range failed'},index=[0]))
                continue
        else:
            print(temp_all_pitching_stats_fn + " already exists")
            continue

    else:
        filenameFormatNoExists.append(filenameFormatNoExists)

print(filenameFormatNoExists)
print(pitching_stats_range_fails)
print(len(pitching_stats_range_fails))
'''
date_string = "2010-04-05"
temp_all_pitching_stats = pitching_stats_range(prior_date_formatted, date.strftime("%Y-%m-%d"))
print(temp_all_pitching_stats[temp_all_pitching_stats['Name'] == 'Roy Halladay'])
'''

In [ ]:
print("run all_batting_stats")
everyPitchFilenames = []
path = "./"
everyPitchPattern = r"\d{4}_every_pitch_\d{2}_\d{2}\.pkl$"  # Match all game_stats file

currentYear = "0"
for filename in os.listdir(path):
    if re.search(everyPitchPattern, filename):
        if currentYear == filename[:4]:
            everyPitchFilenames.append(filename)
        else:
            currentYear = filename[:4]

print("everyPitchFilenames")
print(everyPitchFilenames)
print(len(everyPitchFilenames))
filenameFormatNoExists = []
batting_stats_range_fails = []

for filename in everyPitchFilenames:
    # Define the date in YYYY-MM-DD format

    # Extract year, month, and day from the filename using regular expressions
    match = re.search(r'(\d{4})_every_pitch_(\d{2})_(\d{2})\.pkl', filename)
    if match:
        year = match.group(1)
        month = match.group(2)
        day = match.group(3)

        # Create a datetime object
        date_obj = datetime(int(year), int(month), int(day))

        # Format the date as "YYYY-MM-DD"
        date_string = date_obj.strftime("%Y-%m-%d")

        # Convert the date string to a datetime object
        date = datetime.strptime(date_string, "%Y-%m-%d")
        # Calculate the date 30 days prior
        prior_date = date - timedelta(days=30)
        # Format the prior_date as YYYY-MM-DD
        prior_date_formatted = prior_date.strftime("%Y-%m-%d")
        print(date_string)
        print(prior_date_formatted)

        temp_all_batting_stats=pd.DataFrame([])
        temp_all_batting_stats_fn = filename.replace("_every_pitch_", "_all_batting_stats_")
        temp_all_batting_stats_pickle_in = pd.DataFrame([])
        try:
            temp_all_batting_stats_pickle_in = open(temp_all_batting_stats_fn,"rb")
            temp_all_batting_stats_pickle_in=pickle.load(temp_all_batting_stats_pickle_in)
        except:
            print(temp_all_batting_stats_fn + " not found, creating it")
            temp_all_batting_stats_pickle_in = pd.DataFrame([])
        if len(temp_all_batting_stats_pickle_in) == 0:
            try:
                time.sleep(5)
                temp_all_batting_stats = batting_stats_range(prior_date_formatted, date.strftime("%Y-%m-%d"))
            except Exception as e:
                print('batting_stats_range failed ',date.strftime("%Y-%m-%d"), e, prior_date_formatted)
                batting_stats_range_fails.append(pd.DataFrame({'Date':date,'prior_date_formatted':prior_date_formatted,'exception':e,'Reason':'batting_stats_range failed'},index=[0]))
                continue
            if len(temp_all_batting_stats)>0:
                with open(temp_all_batting_stats_fn, 'wb') as f:  # open a text file
                    pickle.dump(temp_all_batting_stats, f) # serialize the data frame
            elif len(temp_all_batting_stats)==0:
                print('No temp_all_batting_stats')
                batting_stats_range_fails.append(pd.DataFrame({'Date':date,'Reason':'batting_stats_range failed'},index=[0]))
                continue
            else:
                batting_stats_range_fails.append(pd.DataFrame({'Date':date,'Reason':'batting_stats_range failed'},index=[0]))
                continue
        else:
            print(temp_all_batting_stats_fn + " already exists")
            continue

    else:
        filenameFormatNoExists.append(filenameFormatNoExists)

print(filenameFormatNoExists)
print(batting_stats_range_fails)
print(len(batting_stats_range_fails))
'''
date_string = "2010-04-05"
temp_all_batting_stats = batting_stats_range(prior_date_formatted, date.strftime("%Y-%m-%d"))
print(temp_all_batting_stats[temp_all_batting_stats['Name'] == 'Roy Halladay'])
'''

In [ ]:
print("run statcast_pitcher")
listOfGameStatsFilenames = []
path = "./"
gameStatsPattern = r"\d{4}_game_stats_\d{2}_\d{2}_\d+\.pkl$"  # Match all game_stats file

for filename in os.listdir(path):
    if re.search(gameStatsPattern, filename):
        listOfGameStatsFilenames.append(filename)
print("listOfGameStatsFilenames length")
print(len(listOfGameStatsFilenames))

statcast_pitcher_fails = []

for fileName in listOfGameStatsFilenames:
    game_stats = open(fileName,"rb")
    game_stats=pickle.load(game_stats)

    # Harrison check
    # Regular expression pattern to match consecutive digits before ".pkl"
    patternForGameNumber = r'(\d+)\.pkl$'

    # Search for the pattern in the filename
    gameMatch = re.search(patternForGameNumber, fileName)

    game = ""

    # If a match is found, extract the consecutive digits
    if gameMatch:
        game = gameMatch.group(1)
    else:
        print("No match found for game number in file name ", fileName)
        continue
            
    year = fileName[:4]
    month = fileName[16:18]
    day = fileName[19:21]


    # Create a datetime object
    date_obj = datetime(int(year), int(month), int(day))

    # Format the date as "YYYY-MM-DD"
    date_string = date_obj.strftime("%Y-%m-%d")

    # Convert the date string to a datetime object
    date = datetime.strptime(date_string, "%Y-%m-%d")
    # Calculate the date 30 days prior
    prior_date = date - timedelta(days=30)
    # Format the prior_date as YYYY-MM-DD
    prior_date_formatted = prior_date.strftime("%Y-%m-%d")


    temp_home_statcast_pitcher_fn = fileName[:4] + "_statcast_pitcher_" + fileName[16:18] + "_" + fileName[19:21] + "_" + game + "_home.pkl"
    temp_away_statcast_pitcher_fn = fileName[:4] + "_statcast_pitcher_" + fileName[16:18] + "_" + fileName[19:21] + "_" + game + "_away.pkl"

    temp_home_statcast_pitcher_pickle_in = pd.DataFrame([])
    temp_away_statcast_pitcher_pickle_in = pd.DataFrame([])
    temp_home_statcast_pitcher = pd.DataFrame([])
    temp_away_statcast_pitcher = pd.DataFrame([])
    try:
        temp_home_statcast_pitcher_pickle_in = open(temp_home_statcast_pitcher_fn,"rb")
        temp_home_statcast_pitcher_pickle_in=pickle.load(temp_home_statcast_pitcher_pickle_in)
        temp_away_statcast_pitcher_pickle_in = open(temp_away_statcast_pitcher_fn,"rb")
        temp_away_statcast_pitcher_pickle_in=pickle.load(temp_away_statcast_pitcher_pickle_in)
    except:
        print(temp_home_statcast_pitcher_fn + "_" + temp_away_statcast_pitcher_fn + " not found, creating it")
        temp_home_statcast_pitcher_pickle_in = pd.DataFrame([])
        temp_away_statcast_pitcher_pickle_in = pd.DataFrame([])

    if len(temp_home_statcast_pitcher_pickle_in) == 0 or len(temp_away_statcast_pitcher_pickle_in) == 0:
        try:
            home_counter=0
            away_counter=0
            l=game_stats.pitcher.value_counts().keys()
            #determine 'starter' by who threw the most pitches. Normally this would simply be the
            #pitchers who were in the first inning but with the rise of 'bullpenning' this is a work-around
            for pitcher in l:
                m=game_stats[game_stats.pitcher==pitcher]
                m.reset_index(drop=True, inplace=True)
                if home_counter==0 and m.inning_topbot[0]=='Top':
                    home_starter_id=pitcher
                    home_counter+=1
                elif away_counter==0 and m.inning_topbot[0]=='Bot':
                    away_starter_id=pitcher
                    away_counter=+1
                else:
                    None

            print(prior_date_formatted)
            print(date.strftime("%Y-%m-%d"))
            print(int(home_starter_id))
            print(int(away_starter_id))
            temp_home_statcast_pitcher=statcast_pitcher(prior_date_formatted, date.strftime("%Y-%m-%d"),int(home_starter_id))
            temp_away_statcast_pitcher=statcast_pitcher(prior_date_formatted, date.strftime("%Y-%m-%d"),int(away_starter_id))
        except:
            print('statcast_pitcher failed ',date, str(game))
            statcast_pitcher_fails.append(pd.DataFrame({'game#':game,'Date':date,
                                             'Reason':'statcast_pitcher failed'},index=[0]))
            continue
        if len(temp_home_statcast_pitcher)>0 and len(temp_away_statcast_pitcher)>0:
            with open(temp_home_statcast_pitcher_fn, 'wb') as f:  # open a text file
                pickle.dump(temp_home_statcast_pitcher, f) # serialize the data frame
            with open(temp_away_statcast_pitcher_fn, 'wb') as f:  # open a text file
                pickle.dump(temp_away_statcast_pitcher, f) # serialize the data frame
        elif len(temp_home_statcast_pitcher)==0 or len(temp_away_statcast_pitcher)==0:
            print('No home_statcast_pitcher or away_statcast_pitcher')
            statcast_pitcher_fails.append(pd.DataFrame({'game#':game,'Date':date,
                                             'Reason':'statcast_pitcher failed'},index=[0]))
            continue
        else:
            statcast_pitcher_fails.append(pd.DataFrame({'game#':game,'Date':date,
                                             'Reason':'statcast_pitcher failed'},index=[0]))
            continue

    else:
        print(temp_home_statcast_pitcher_fn + " and " + temp_away_statcast_pitcher_fn + " already exists")
        continue

print(len(statcast_pitcher_fails))
print(statcast_pitcher_fails)


In [ ]:
temp_all_batting_stats_pickle_in = open("2010_statcast_pitcher_04_05_263813_away.pkl","rb")
temp_all_batting_stats_pickle_in=pickle.load(temp_all_batting_stats_pickle_in)
print(temp_all_batting_stats_pickle_in)

In [ ]:
#temp_all_batting_stats = batting_stats_range("2010-04-20", "2010-04-22")
#print(temp_all_batting_stats)
temp_all_batting_stats_pickle_in = open("2010_all_batting_stats_04_06.pkl","rb")
temp_all_batting_stats_pickle_in=pickle.load(temp_all_batting_stats_pickle_in)
print(temp_all_batting_stats_pickle_in)

In [ ]:
# check for failed requests and then retry them
failedRequests = [
#"2010-06-15",
#"2013-07-13",
#"2014-08-12",
#"2014-08-18",
#"2014-08-19",
#"2014-08-20",
#"2014-08-21",
#"2014-08-22",
#"2014-08-23",
#"2014-08-24",
#"2014-08-25",
#"2014-09-13",
#"2014-09-24",
#"2014-09-25",
"2014-09-29",
#"2015-05-08",
#"2015-06-23",
#"2016-05-14"
]

for day in failedRequests:
    temp=pd.DataFrame([])
    try:
        temp=statcast(day,day)
    except:
        print(day)
    if len(temp)>0:
        dayDf=pd.DataFrame(temp)
        fileName = day[:4] + '_every_pitch_' + day[5:7] + '_' + day[-2:] + '.pkl'
        with open(fileName, 'wb') as f:  # open a text file
            pickle.dump(dayDf, f) # serialize the data frame


In [ ]:
failedRequestsFiles = []
for key in datesDict:
    print(key)
    for date in datesDict[key]:
        date = str(date)
        dayDf=pd.DataFrame([])
        modMonth = date[:-2]
        if len(modMonth) == 1:
            modMonth = "0" + modMonth
        fileName = key + '_every_pitch_' + modMonth + '_' + date[-2:] + '.pkl'
        my_file = Path(fileName)
        if not my_file.is_file():
            failedRequestsFiles.append(fileName)
print(failedRequestsFiles)
print(len(failedRequestsFiles))
for day in failedRequestsFiles:
    temp=pd.DataFrame([])
    try:
        tempDay = day[:4] + "-" + day[-9:-7] + "-" + day[-6:-4]
        print("tempDay", tempDay)
        temp=statcast(tempDay,tempDay)
    except Exception as error:
        print("error", error)
    if len(temp)>0:
        dayDf=pd.DataFrame(temp)
        fileName = day[:4] + '_every_pitch_' + day[-9:-7] + '_' + day[-6:-4] + '.pkl'
        with open(fileName, 'wb') as f:  # open a text file
            pickle.dump(dayDf, f) # serialize the data frame


In [ ]:
'''
x=list(range(0))
all_players=playerid_lookup('WAINWRIGHT', 'adam')
all_players=pd.DataFrame(all_players)
all_players=all_players[all_players.key_fangraphs!=-1]
print(all_players)
'''
x=list(range(0,1000000))
all_players=playerid_reverse_lookup(x)
all_players=pd.DataFrame(all_players)
all_players=all_players[all_players.key_fangraphs!=-1]
print(all_players)
with open("wrangle_data_all_players.pkl", 'wb') as f:  # open a text file
    pickle.dump(all_players, f) # serialize the data frame
print("all_players written to pickle file")